#### Paso 1.
##### Realice la limpieza de los datasets:
- Renombre la columna "default payment next month" a "default".
- Remueva la columna "ID".
- Elimine los registros con informacion no disponible.
- Para la columna EDUCATION, valores > 4 indican niveles superiores  de educación, agrupe estos valores en la categoría "others".
- Renombre la columna "default payment next month" a "default"
- Remueva la columna "ID".

In [1]:
import pandas as pd

In [2]:
df_train=pd.read_csv('../files/input/train_data.csv.zip',index_col=False,compression='zip')
df_test=pd.read_csv('../files/input/test_data.csv.zip',index_col=False,compression='zip')

In [3]:
df_train.rename(columns={'default payment next month':'default'},inplace=True)
df_test.rename(columns={'default payment next month':'default'},inplace=True)
# print(df_train.columns)
df_train.drop(columns=['ID'],inplace=True)
df_test.drop(columns=['ID'],inplace=True)
suma_nulos_train=df_train.isnull().sum()
suma_nulos_test=df_test.isnull().sum()
# print(suma_nulos_train)
suma_na_train=df_train.isna().sum()
suma_na_test=df_test.isna().sum()
# print(suma_na_train)
# print(suma_na_test)

# print(df_train.value_counts("EDUCATION"))
df_train['EDUCATION']=df_train['EDUCATION'].apply(lambda x: 4 if x > 4 else x)
df_test['EDUCATION']=df_test['EDUCATION'].apply(lambda x: 4 if x > 4 else x)

#se seleccionan 
df_train=df_train.loc[df_train['EDUCATION']!=0]
df_test=df_test.loc[df_test['EDUCATION']!=0]

df_train=df_train.loc[df_train['MARRIAGE']!=0]
df_test=df_test.loc[df_test['MARRIAGE']!=0]

In [4]:
print(df_train['MARRIAGE'].value_counts())
print(df_test['MARRIAGE'].value_counts())
#df_train['SEX'].value_counts()
#df_test['SEX'].value_counts()

MARRIAGE
2    11226
1     9502
3      225
Name: count, dtype: int64
MARRIAGE
2    4728
1    4153
3      98
Name: count, dtype: int64


#### Paso 2.
##### Divida los datasets en x_train, y_train, x_test, y_test.

In [5]:
X_train = df_train.drop(columns=["default"]) 
y_train = df_train["default"]               

X_test = df_test.drop(columns=["default"])  
y_test =df_test["default"]

#### Paso 3.
##### Cree un pipeline para el modelo de clasificación. Este pipeline debe contener las siguientes capas:
- Transforma las variables categoricas usando el método one-hot-encoding.
- Escala las demas variables al intervalo [0, 1].
- Selecciona las K mejores caracteristicas.
- Ajusta un modelo de regresion logistica.

In [37]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline



# numerical_features= ['LIMIT_BAL', 'AGE', 'PAY_0', 'PAY_2',
#        'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
#        'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
#        'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6' ]

#Se define el preprocesaror para aplicar el one-hot encoding a las variables categoricas
transformer = ColumnTransformer( transformers=[ ('ohe', OneHotEncoder(), ['SEX', 'EDUCATION','MARRIAGE'])
    ],
    remainder = MinMaxScaler())

selectkbest = SelectKBest(score_func=f_classif)
# Crea un objeto pipeline
pipeline = Pipeline(steps=[('transformer', transformer),
                           ('estimator', selectkbest),
                           ('model', LogisticRegression(class_weight='balanced', solver='liblinear',max_iter=1000))])


#### Paso 4.
- Optimice los hiperparametros del pipeline usando validación cruzada.
- Use 10 splits para la validación cruzada. Use la función de precision balanceada para medir la precisión del modelo.

In [46]:
from sklearn.model_selection import  GridSearchCV
from sklearn.metrics import balanced_accuracy_score

# param_grid = {
#         'model__penalty': ['l2', None], #12
#         'model__solver': ['lbfgs'], #lbfgs
#         'model__C': [0.01, 0.1, 1, 10, 100] #1
#     }

param_grid = {
        'model__penalty': ['l2'],  # Regularización Ridge
        'model__solver': ['lbfgs', 'newton-cg', 'sag'],  # Solvers compatibles con l2
        'model__C': [0.01, 0.1, 1, 10, 100],  # Control de regularización inversa
        'model__class_weight': ['balanced']  # Ponderación automática para balancear clases
    }
#     {
#         'model__penalty': ['l1'],  # Regularización Lasso
#         'model__solver': ['liblinear', 'saga'],  # Solvers compatibles con l1
#         'model__C': [0.01, 0.1, 1, 10, 100],
#         'model__class_weight': ['balanced']
#     },
#     {
#         'model__penalty': ['elasticnet'],  # Regularización combinada L1 y L2
#         'model__solver': ['saga'],  # Saga admite elasticnet
#         'model__C': [0.01, 0.1, 1, 10],
#         'model__l1_ratio': [0.1, 0.5, 0.9],  # Peso entre L1 y L2
#         'model__class_weight': ['balanced']
#     }
# ]

# Crear el objeto GridSearchCV con validación cruzada de 10 divisiones y precisión balanceada 
grid_search = GridSearchCV(
    estimator = pipeline,
    param_grid = param_grid,
    scoring = "balanced_accuracy", 
    cv=10,
    n_jobs=-1
    )

# Entrenar el modelo con GridSearchCV 
grid_search.fit(X_train, y_train)

c:\Users\Sofia\Desktop\predictiva\20204-2-LAB-02-prediccion-del-default-usando-logreg-SofiaRosas99\.venv\Lib\site-packages\sklearn\compose\_column_transformer.py:1651: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('transformer',
                                        ColumnTransformer(remainder=MinMaxScaler(),
                                                          transformers=[('ohe',
                                                                         OneHotEncoder(),
                                                                         ['SEX',
                                                                          'EDUCATION',
                                                                          'MARRIAGE'])])),
                                       ('estimator', SelectKBest()),
                                       ('model',
                                        LogisticRegression(class_weight='balanced',
                                                           max_iter=1000,
                                                           solver='liblinear'))]),
             n_jobs=-1,
             param_grid=[{'model__C': [0.01, 0.1, 1, 10, 100],
                          'mod...
                          'model__solver': ['lbfgs', 'newton-cg', 'sag']},
                         {'model__C': [0.01, 0.1, 1, 10, 100],
                          'model__class_weight': ['balanced'],
                          'model__penalty': ['l1'],
                          'model__solver': ['liblinear', 'saga']},
                         {'model__C': [0.01, 0.1, 1, 10],
                          'model__class_weight': ['balanced'],
                          'model__l1_ratio': [0.1, 0.5, 0.9],
                          'model__penalty': ['elasticnet'],
                          'model__solver': ['saga']}],
             scoring='balanced_accuracy')

In [47]:
# Obtener los mejores parámetros y el mejor modelo 
best_params = grid_search.best_params_ #coge los mejores parametros del estimador
model = grid_search.best_estimator_  #en este caso solo se tiene un estimador 
best_score = grid_search.best_score_
print(best_params)

{'model__C': 0.01, 'model__class_weight': 'balanced', 'model__penalty': 'l1', 'model__solver': 'saga'}


#### Paso 5.
- Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz".
- Recuerde que es posible guardar el modelo comprimido usanzo la libreria gzip.

In [48]:
import gzip
import pickle
import os
# Crear carpeta si no existe
output_dir = '../files/models'
os.makedirs(output_dir, exist_ok=True)

compressed_model_path = '../files/models/model.pkl.gz'

with gzip.open(compressed_model_path, 'wb') as f:
    pickle.dump(model, f)

#### Paso 6.
- Calcule las metricas de precision, precision balanceada, recall, y f1-score para los conjuntos de entrenamiento y prueba.
- Guardelas en el archivo files/output/metrics.json. Cada fila del archivo es un diccionario con las metricas de un modelo.
- Este diccionario tiene un campo para indicar si es el conjunto de entrenamiento o prueba. Por ejemplo:

{'dataset': 'train', 'precision': 0.8, 'balanced_accuracy': 0.7, 'recall': 0.9, 'f1_score': 0.85}

{'dataset': 'test', 'precision': 0.7, 'balanced_accuracy': 0.6, 'recall': 0.8, 'f1_score': 0.75}

In [49]:
from sklearn.metrics import precision_score, recall_score, f1_score
import json

y_train_pred = model.predict(X_train) 
y_test_pred = model.predict(X_test)

# Calcular las métricas para el conjunto de entrenamiento, en la primera va el nombre del dataset y en la segunda el train 
train_metrics = { 'type': "metrics",
    'dataset': 'train', 
                'precision': float(precision_score(y_train, y_train_pred, average='weighted')),
                'balanced_accuracy': float(balanced_accuracy_score(y_train, y_train_pred)), 
                'recall': float(recall_score(y_train, y_train_pred, average='weighted')), 
                'f1_score': float(f1_score(y_train, y_train_pred, average='weighted')) }

# Calcular las métricas para el conjunto de prueba 
test_metrics = { 'type': "metrics",
    'dataset': 'test', 
                'precision': float(precision_score(y_test, y_test_pred, average='weighted')), 
                'balanced_accuracy': float(balanced_accuracy_score(y_test, y_test_pred)),
                'recall': float(recall_score(y_test, y_test_pred, average='weighted')), 
                'f1_score': float(f1_score(y_test, y_test_pred, average='weighted')) }

print("Metricas de entrenamiento")
print(json.dumps(train_metrics, indent=4))
print("Metricas de prueba")
print(json.dumps(test_metrics, indent=4))


Metricas de entrenamiento
{
    "type": "metrics",
    "dataset": "train",
    "precision": 0.7809880910286923,
    "balanced_accuracy": 0.6921492540817165,
    "recall": 0.7730635231231805,
    "f1_score": 0.7766478475980322
}
Metricas de prueba
{
    "type": "metrics",
    "dataset": "test",
    "precision": 0.7895136444304646,
    "balanced_accuracy": 0.6935665965291654,
    "recall": 0.7774807885065153,
    "f1_score": 0.7827440609189583
}


In [50]:
#############  paso 7
from sklearn.metrics import confusion_matrix
# Matriz de confusión para el conjunto de entrenamiento
cm_train = confusion_matrix(y_train, y_train_pred)

# Matriz de confusión para el conjunto de prueba
cm_test = confusion_matrix(y_test, y_test_pred)
# Crear la matriz de confusión formateada como diccionario
metrics = []

# Matriz de confusión para entrenamiento
metrics = [
    {
        'type': 'cm_matrix',
        'dataset': 'train',
        'true_0': {"predicted_0": int(cm_train[0][0]), "predicted_1": int(cm_train[0][1])},
        'true_1': {"predicted_0": int(cm_train[1][0]), "predicted_1": int(cm_train[1][1])}
    },
    {
        'type': 'cm_matrix',
        'dataset': 'test',
        'true_0': {"predicted_0": int(cm_test[0][0]), "predicted_1": int(cm_test[0][1])},
        'true_1': {"predicted_0": int(cm_test[1][0]), "predicted_1": int(cm_test[1][1])}
    }
    
]

# Crear carpeta si no existe
output_dir = '../files/output'
os.makedirs(output_dir, exist_ok=True)

# Guardar las métricas en un archivo JSON
output_path = '../files/output/metrics.json'
with open(output_path, 'w') as f:  # Usar 'w' para comenzar con un archivo limpio
        f.write(json.dumps(train_metrics) + '\n')
        f.write(json.dumps(test_metrics) + '\n')
        for metric in metrics:
            f.write(json.dumps(metric) + '\n')


print(f"Métricas guardadas en {output_path}")

Métricas guardadas en ../files/output/metrics.json
